In [8]:
import pandas as pd
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
import numpy as np
import re

[nltk_data] Downloading package punkt to /Users/letriluan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/letriluan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df = pd.read_csv('/Users/letriluan/Downloads/NLP/cleaned_data.csv', encoding="latin1")
df

,id,author,date,year,month,topic,article
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS Islamic State drive ancient city Palmyra...
1,17292,Andy Newman,31/12/2016,2016,12,art,angel everywhere Mu iz family apartment Bronx ...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,finally Second Avenue subway open New York Cit...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON time Republicans tumultuous decade ...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,Megyn Kelly shift Fox News NBC host daily dayt...
...,...,...,...,...,...,...,...
995,18460,Gerry Mullany,14/03/2017,2017,3,accidents,HONG KONG Hundreds pilot whale swam shallow Ne...
996,18461,Rory Smith,10/02/2017,2017,2,sports,NICE France Riv accept compliment reject compa...
997,18462,Jack Ewing,9/02/2017,2017,2,business,FRANKFURT Germans never really warm euro may a...
998,18463,Scott Cacciola,10/02/2017,2017,2,sports,Charles Oakley strong feeling compe ticket fre...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1000 non-null   int64 
 1   author   994 non-null    object
 2   date     1000 non-null   object
 3   year     1000 non-null   object
 4   month    1000 non-null   object
 5   topic    1000 non-null   object
 6   article  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [11]:
df.describe()

,id
count,1000.00000
mean,17878.53200
std,341.50184
min,17283.00000
25%,17582.75000
50%,17881.50000
75%,18182.25000
max,18465.00000


In [12]:
df.isnull().sum()

id         0
author     6
date       0
year       0
month      0
topic      0
article    0
dtype: int64

**Pre-process**

In [104]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import neuralcoref

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

def clean_text1(text):
    tokenizer = RegexpTokenizer(r'\b\w+\b') 
    tokens = tokenizer.tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    doc = nlp(" ".join(filtered_tokens))
    lemmatized_tokens = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]
    
    return " ".join(lemmatized_tokens)

df_new = df.copy()
df_new["article"] = df_new["article"].apply(clean_text)


In [13]:
df_new = df.copy()

**Coreference Resolution utility**

In [14]:
def resolve_coreferences(text):
    doc = nlp(text)
    if doc._.has_coref:
        return doc._.coref_resolved
    return text

**Text matching utility**

In [15]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def find_most_relevant_sentence(question, article_text):
    doc = nlp(article_text)
    sentences = [sent.text for sent in doc.sents]
    question_embedding = model.encode(question)
    sentence_embeddings = model.encode(sentences)

    similarities = util.pytorch_cos_sim(question_embedding, sentence_embeddings).squeeze()
    most_similar_index = similarities.argmax().item()
    confidence = similarities[most_similar_index].item()
    
    return sentences[most_similar_index], confidence


In [16]:
nlp = spacy.load("en_core_web_sm")
def extract_relevant_snippets(question, relevant_sentence):
    doc = nlp(relevant_sentence)
    question_doc = nlp(question)

    target_label = 'PERSON'  # Default to PERSON
    
    if any(word in question.lower() for word in ['who', 'name']):
        target_label = 'PERSON'
    elif any(word in question.lower() for word in ['when', 'date', 'year', 'time']):
        target_label = 'DATE'
    elif any(word in question.lower() for word in ['where', 'city', 'country', 'place', 'location']):
        target_label = 'GPE'
    elif any(word in question.lower() for word in ['what', 'company', 'organization']):
        target_label = 'ORG'
    elif 'how many' in question.lower():
        target_label = 'CARDINAL' 

    entities = {}
    for ent in doc.ents:
        if ent.label_ == target_label:
            if ent.text in entities:
                entities[ent.text] += 1
            else:
                entities[ent.text] = 1

    if entities:
        sorted_entities = sorted(entities.items(), key=lambda item: (-item[1], relevant_sentence.index(item[0])))
        return sorted_entities[0][0]

    return "No relevant information found."


In [17]:
def answer_question_from_article(article_id, question, df):
    try:
        article_text = df.loc[df['id'] == article_id, 'article'].values[0]
    except IndexError:
        return "Article not found."

    relevant_sentence, confidence = find_most_relevant_sentence(question, article_text)
    
    confidence_threshold = 0.5
    if confidence < confidence_threshold:
        return "High confidence answer not found."
    
    answer_snippet = extract_relevant_snippets(question, relevant_sentence)
    return answer_snippet, confidence

article_id = 17574  
question = "Who is the vice chairman of Samsung?"
answer = answer_question_from_article(article_id, question, df_new)
print("Answer snippet:", answer)

Answer snippet: ('Jay Lee', 0.5828641653060913)


**Test utility**

In [109]:
def calculate_mrr(queries, answers, article_ids, df):
    reciprocal_ranks = []
    for query, correct_answer, article_id in zip(queries, answers, article_ids):
        predicted_answer, _ = answer_question_from_article(article_id, query, df)
        if predicted_answer == correct_answer:
            reciprocal_ranks.append(1)  # Correct answer was ranked first
        else:
            reciprocal_ranks.append(0)  # Correct answer was not found
    return sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0

def calculate_map(queries, answers, article_ids, df):
    average_precisions = []
    for query, correct_answer, article_id in zip(queries, answers, article_ids):
        predicted_answer, _ = answer_question_from_article(article_id, query, df)
        if predicted_answer == correct_answer:
            average_precisions.append(1)  # Correct answer, precision of 1
        else:
            average_precisions.append(0)  # Incorrect answer
    return sum(average_precisions) / len(average_precisions) if average_precisions else 0


In [110]:
# Define a more complete set of test questions and corresponding answers based on your dataset structure
questions = [
    "Who is the vice chairman of Samsung?",
    "When will the vice chairman of Samsung be questioned?",
    "How many pilot whales that swam into a shallow New Zealand bay died overnight?",
    "How many rescuers tried frantically to send the pilot whales back out to sea?",
    "Where is has one of the highest rates of whale strandings?"
]
answers = [
    "Jay Lee",
    "Thursday",
    "Hunders",
    "500",
    "New Zealand"
]
article_ids = [
    17574,  # Assume all questions pertain to details in the article with ID 17574
    17574,
    18460,
    18460,
    18460
]

# Assuming df_new is properly loaded and structured as per your system requirements
mrr_score = calculate_mrr(questions, answers, article_ids, df_new)
map_score = calculate_map(questions, answers, article_ids, df_new)

print("MRR Score:", mrr_score)
print("MAP Score:", map_score)


MRR Score: 0.8
MAP Score: 0.8


**Interaction**

In [111]:
def user_interaction():
    while True:
        article_id = input("Enter the article ID or type 'quit' to exit: ")
        if article_id.lower() == 'quit':
            break
        print( 'Article ID: ', article_id)
        question = input("Enter your question: ")

        if question.lower() == 'quit':
            break
        print('Your question: ', question)
        try:
            article_id = int(article_id)
            answer = answer_question_from_article(article_id, question, df_new)
            print("Answer:", answer)
        except ValueError:
            print("Invalid article ID. Please enter a numeric ID.")

# To run the interaction
user_interaction()

Article ID:  17574
Your question:  Who run the Samsung effectively?
Answer: High confidence answer not found.
Article ID:  17574
Your question:  Who is vice the chairman of Samsung?
Answer: ('Jay Lee', 0.5779139995574951)
Article ID:  17574
Your question:  Who is vice the chairman of Samsung?
Answer: ('Jay Lee', 0.5779139995574951)


In [133]:
user_interaction()  #Tranformers based

Article ID:  15
Your question:  who I am ?
Answer: Article not found.
Article ID:  17574
Your question:  Who run the Samsung effectively?
Answer: High confidence answer not found.
Article ID:  17574
Your question:  Who is vice the chairman of Samsung?
Answer: ('Jay Lee', 0.5779139995574951)
Article ID:  17574
Your question:  When the vice Chaiman of Samsung will be questioned?
Answer: ('Thursday', 0.5326932072639465)


In [135]:
user_interaction()  #Tranformers based

Article ID:  17574
Your question:  When Jay Lee will be questioned?
Answer: ('November', 0.5102343559265137)


In [137]:
user_interaction() #Tranformers based

Article ID:  18460
Your question:  How many pilot whales that swam into a shallow New Zealand bay died overnight? 
Answer: ('Hundreds', 0.8101339936256409)
Article ID:  18460
Your question:  How many rescuers tried frantically to send the pilot whales back out to sea?
Answer: ('500', 0.7037572264671326)
Article ID:  18460
Your question:  Where is has one of the highest rates of whale strandings?
Answer: ('New Zealand', 0.6138548851013184)


**BERT**

In [138]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def answer_question_bert(question, context):
    """Function to answer questions using BERT directly from the context."""
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs, return_dict=True)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert the tokens back to the original words
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer

def answer_question_bert(question, context):
    """Function to answer questions using BERT directly from the context, including confidence score."""
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    with torch.no_grad(): 
        outputs = model(**inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

    start_probs = softmax(answer_start_scores, dim=-1)
    end_probs = softmax(answer_end_scores, dim=-1)
    answer_start = torch.argmax(start_probs)
    answer_end = torch.argmax(end_probs) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

def answer_question_from_article(article_id, question, df):
    """Retrieve an article by ID and use BERT to answer a question based on the article's text, including confidence."""
    try:
        article_text = df.loc[df['id'] == article_id, 'article'].values[0]
    except IndexError:
        return "Article not found."

    answer = answer_question_bert(question, article_text)
    return answer

article_id = 18460
question = "Where is has one of the highest rates of whale strandings?"
answer = answer_question_from_article(article_id, question, df_new)
print("Answer snippet:", answer)

Answer snippet: new zealand


In [139]:
user_interaction() #BERT model

Article ID:  15
Your question:  who I am?
Answer: Article not found.
Article ID:  17574
Your question:  Who run the Samsung effectively?
Answer: mr lee
Article ID:  17574
Your question:  Who is vice the chairman of Samsung?
Answer: jay lee
Article ID:  17574
Your question:  When Jay Lee will be questioned? 
Answer: thursday


In [140]:
user_interaction() #BERT model

Article ID:  18460
Your question:  How many pilot whales that swam into a shallow New Zealand bay died overnight?
Answer: hundreds
Article ID:  18460
Your question:  How many rescuers tried frantically to send the pilot whales back out to sea?
Answer: 500
Article ID:  18460
Your question:  Where is has one of the highest rates of whale strandings?
Answer: new zealand


In [ ]:
import pandas as pd
import spacy
import nltk
nltk.download('punkt')
import numpy as np
import streamlit as st
from sentence_transformers import SentenceTransformer, util

nlp = spacy.load('en_core_web_sm')
df = pd.read_csv('cleaned_data.csv', encoding="latin1")
model = SentenceTransformer('all-MiniLM-L6-v2')

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def find_most_relevant_sentence(question, article_text):
    doc = nlp(article_text)
    sentences = [sent.text for sent in doc.sents]
    question_embedding = model.encode(question)
    sentence_embeddings = model.encode(sentences)

    similarities = util.pytorch_cos_sim(question_embedding, sentence_embeddings).squeeze()
    most_similar_index = similarities.argmax().item()
    confidence = similarities[most_similar_index].item()
    
    return sentences[most_similar_index], confidence


nlp = spacy.load("en_core_web_sm")
def extract_relevant_snippets(question, relevant_sentence):
    doc = nlp(relevant_sentence)
    question_doc = nlp(question)

    target_label = 'PERSON'  # Default to PERSON
    
    if any(word in question.lower() for word in ['who', 'name']):
        target_label = 'PERSON'
    elif any(word in question.lower() for word in ['when', 'date', 'year', 'time']):
        target_label = 'DATE'
    elif any(word in question.lower() for word in ['where', 'city', 'country', 'place', 'location']):
        target_label = 'GPE'
    elif any(word in question.lower() for word in ['what', 'company', 'organization']):
        target_label = 'ORG'
    elif 'how many' in question.lower():
        target_label = 'CARDINAL' 

    entities = {}
    for ent in doc.ents:
        if ent.label_ == target_label:
            if ent.text in entities:
                entities[ent.text] += 1
            else:
                entities[ent.text] = 1

    if entities:
        sorted_entities = sorted(entities.items(), key=lambda item: (-item[1], relevant_sentence.index(item[0])))
        return sorted_entities[0][0]

    return "No relevant information found."


def answer_question_from_article(article_id, question, df):
    try:
        article_text = df.loc[df['id'] == article_id, 'article'].values[0]
    except IndexError:
        return "Article not found."

    relevant_sentence, confidence = find_most_relevant_sentence(question, article_text)
    
    confidence_threshold = 0.5
    if confidence < confidence_threshold:
        return "High confidence answer not found."
    
    answer_snippet = extract_relevant_snippets(question, relevant_sentence)
    return answer_snippet, confidence

def main():
    st.title("Question Answering System")
    question = st.text_input("Enter your question:")
    article_id = st.number_input("Enter the article ID:", value=0, step=1)

    if st.button("Get Answer"):
        answer = answer_question_from_article(article_id, question, df)
        st.write("Answer:", answer)

if __name__ == "__main__":
    main()